In [1]:
import pandas as pd
import datetime as dt

gbq_prj = 'data-lake-prod-223818'
pd.set_option('display.max_columns', 500)

if dt.date.today().month == 1:
    year = dt.date.today().year-1
    month = 12
else:
    year = dt.date.today().year
    month = dt.date.today().month-1

if len(str(month)) == 1:
    month = str(0) + str(month)
else:
    month = month

date_string = str(year)+str(month)

output_file = 'out_files\\{}_exhibit_b.xlsx'.format(date_string)

terms = df['Term'].unique()
ratings = ['AA', 'A', 'B', 'C', 'D', 'E', 'HR']

In [2]:
sql = open('sql/get_vintage_data.sql', 'r')
sql = sql.read()
sql = sql.format(date_string)

In [4]:
df = pd.read_gbq(sql, project_id=gbq_prj, dialect='standard')

In [37]:
df = df[df['OQ'] >= '2012-01-01']

In [6]:
df['Total DQ'] = pd.to_numeric(df['LoanAmount']).values * pd.to_numeric(df['DPD_16']).values
df['OQ'] = df['OriginationQuarter'].str[3:].map(str) + df['OriginationQuarter'].str[:2].map(str)

In [104]:
df['CDR'] = calc_cdr(df)
df['CPR'] = calc_cpr(df)

In [101]:
def calc_cdr(df):
    smm = (df['CO_Balance'] / df['PrevUPB'])
    cdr = 1 - (1 - smm)**12
    return pd.to_numeric(cdr)

In [102]:
def calc_cpr(df):
    smm = (df['FullPaydowns'] + df['VoluntaryExcessPrin']) / (df['PrevUPB'] - df['ScheduledPeriodicPrin'])
    cpr = 1 - (1 - smm)**12
    return pd.to_numeric(cpr)

In [49]:
def numerize_df(df):
    for c in df.columns:
        try:
            df[c] = pd.to_numeric(df[c])
        except:
            pass
    return df

In [324]:
def loan_amount(df):
    df1 = df[(df['CycleCounter'] == 0)]
    df2 = pd.DataFrame(df1.groupby(['OQ'])['LoanAmount'].sum())
    df2 = df2.T
    return df2

In [277]:
def avg_br_rate(writer, df):
              
    for t in terms:
        df1 = df[df['Term'] == t]
        df1 = df1.pivot_table(index='OQ'
                              ,columns='ProsperRating'
                              ,values='AvgBorrowerRate'
                              ,aggfunc='max')
        df1 = df1.reindex_axis(ratings, axis=1)
        if t == 60:
            df1 = df1.drop('HR', axis=1)
        df1.style.format('{:,.2%}')
        df1.to_excel(writer, sheet_name='WTD_AVG_CPN_' + str(t))

In [315]:
def max_cumul_loss(writer, df):
    
    for t in terms:
        df1 = df[(df['Term'] == t)]
        loan_amt = loan_amount(df1)
        pivot1= df1.pivot_table(index='CycleCounter'
                                ,columns='OQ'
                                ,values='CumulativeGrossLosses'
                                ,aggfunc='sum'
                                ,fill_value='')
        pivot2 = df1.pivot_table(index='CycleCounter'
                                ,columns='OQ'
                                ,values='LoanAmount'
                                ,aggfunc='sum'
                                ,fill_value='')
        df2 = numerize_df(pivot1).div(numerize_df(pivot2))
        df2.fillna('', axis=1, inplace=True)
        df2 = pd.concat([loan_amt, df2], sort=True)
        df2 = df2.T.set_index(df2.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
        if not df2.empty:
            df2.to_excel(writer, sheet_name='MCL_' + str(t))

    for t in terms:
        for r in ratings:
            df1 = df[(df['Term'] == t) & (df['ProsperRating'] == r)]
            loan_amt = loan_amount(df1)
            df1 = df1.pivot_table(index='CycleCounter'
                                  ,columns='OQ'
                                  ,values='CumulativeGrossLossesPct'
                                  ,aggfunc='max'
                                  ,fill_value='')
            df1.fillna('', axis=1, inplace=True)
            df1 = pd.concat([loan_amt, df1], sort=True)
            df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
            if not df1.empty:
                df1.to_excel(writer, sheet_name='MCL_' + str(r) + str(t))

In [279]:
def days_past_due(writer, df):

    for t in terms:
        df1 = df[(df['Term'] == t)]
        loan_amt = loan_amount(df1)
        pivot1= df1.pivot_table(index='CycleCounter'
                                ,columns='OQ'
                                ,values='Total DQ'
                                ,aggfunc='sum'
                                ,fill_value='')
        pivot2 = df1.pivot_table(index='CycleCounter'
                                ,columns='OQ'
                                ,values='LoanAmount'
                                ,aggfunc='sum'
                                ,fill_value='')
        df2 = numerize_df(pivot1).div(numerize_df(pivot2))
        df2.fillna('', axis=1, inplace=True)
        df2 = pd.concat([loan_amt, df2], sort=True)
        df2 = df2.T.set_index(df2.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
        if not df2.empty:
            df2.to_excel(writer, sheet_name='DPD_' + str(t))

    for t in terms:
        for r in ratings:
            df1 = df[(df['Term'] == t) & (df['ProsperRating'] == r)]
            loan_amt = loan_amount(df1)
            df1 = df1.pivot_table(index='CycleCounter'
                                  ,columns='OQ'
                                  ,values='DPD_16'
                                  ,aggfunc='max'
                                  ,fill_value='')
            df1.fillna('', axis=1, inplace=True)
            df1 = pd.concat([loan_amt, df1], sort=True)
            df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
            if not df1.empty:
                df1.to_excel(writer, sheet_name='DPD_' + str(r) + str(t))

In [280]:
def end_prin_bal(writer, df):

    for t in terms:
        df1 = df[(df['Term'] == t)]
        loan_amt = loan_amount(df1)
        df1 = df1.pivot_table(index='CycleCounter'
                              ,columns='OQ'
                              ,values='UPB'
                              ,aggfunc='sum'
                              ,fill_value='')
        df1.fillna('', axis=1, inplace=True)
        df1 = pd.concat([loan_amt, df1], sort=True)
        df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
        if not df1.empty:
            df1.to_excel(writer, sheet_name='EOP_' + str(t))

    for t in terms:
        for r in ratings:
            df1 = df[(df['Term'] == t) & (df['ProsperRating'] == r)]
            loan_amt = loan_amount(df1)
            df1 = df1.pivot_table(index='CycleCounter'
                                  ,columns='OQ'
                                  ,values='UPB'
                                  ,aggfunc='sum'
                                  ,fill_value='')
            df1.fillna('', axis=1, inplace=True)
            df1 = pd.concat([loan_amt, df1], sort=True)
            df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
            if not df1.empty:
                df1.to_excel(writer, sheet_name='EOP_' + str(r) + str(t))

In [281]:
def orig_prin_bal(writer, df):

    for t in terms:
        df1 = df[df['Term'] == t]
        df1 = df1.pivot_table(index='OQ'
                              ,columns='ProsperRating'
                              ,values='LoanAmount'
                              ,aggfunc='max')
        df1 = df1.reindex_axis(ratings, axis=1)
        if t == 60:
            df1 = df1.drop('HR', axis=1)
        df1.style.format('${:,.2f}')
        df1.to_excel(writer, sheet_name='OPB_' + str(t))

In [282]:
def cpr(writer, df):
    
    for t in terms:
        df1 = df[(df['Term'] == t)]
        loan_amt= loan_amount(df1)
        df1 = df1.pivot_table(index='CycleCounter'
                              ,columns='OQ'
                              ,values='CPR'
                              ,aggfunc='max'
                              ,fill_value='')
        df1.fillna('', axis=1, inplace=True)
        df1 = pd.concat([loan_amt, df1], sort=True)
        df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
        if not df1.empty:
            df1.to_excel(writer, sheet_name='CPR_' + str(t))

    for t in terms:
        for r in ratings:
            df1 = df[(df['Term'] == t) & (df['ProsperRating'] == r)]
            loan_amt = loan_amount(df1)
            df1 = df1.pivot_table(index='CycleCounter'
                                  ,columns='OQ'
                                  ,values='CPR'
                                  ,aggfunc='max'
                                  ,fill_value='')
            df1.fillna('', axis=1, inplace=True)
            df1 = pd.concat([loan_amt, df1], sort=True)
            df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
            if not df1.empty:
                df1.to_excel(writer, sheet_name='CPR_' + str(r) + str(t))

In [283]:
def cdr(writer, df):
    
    for t in terms:
        df1 = df[(df['Term'] == t)]
        loan_amt = loan_amount(df1)
        df1 = df1.pivot_table(index='CycleCounter'
                              ,columns='OQ'
                              ,values='CDR'
                              ,aggfunc='max'
                              ,fill_value='')
        df1.fillna('', axis=1, inplace=True)
        df1 = pd.concat([loan_amt, df1], sort=True)
        df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
        df1.to_excel(writer, sheet_name='CDR_' + str(t))
        
    for t in terms:
        for r in ratings:
            df1 = df[(df['Term'] == t) & (df['ProsperRating'] == r)]
            loan_amt = loan_amount(df1)
            df1 = df1.pivot_table(index='CycleCounter'
                                  ,columns='OQ'
                                  ,values='CDR'
                                  ,aggfunc='max'
                                  ,fill_value='')
            df1.fillna('', axis=1, inplace=True)
            df1 = pd.concat([loan_amt, df1], sort=True)
            df1 = df1.T.set_index(df1.T['LoanAmount'], append=True).drop(['LoanAmount'], axis=1).swaplevel(0,1).T
            
            if not df1.empty:
                df1.to_excel(writer, sheet_name='CDR_' + str(r) + str(t))

In [326]:
def generate_dt_data():
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    avg_br_rate(writer, df)
    max_cumul_loss(writer, df)
    days_past_due(writer, df)
    end_prin_bal(writer, df)
    orig_prin_bal(writer, df)
    cpr(writer, df)
    cdr(writer, df)
    writer.save()

In [327]:
generate_dt_data()

c:\users\bmauck\appdata\local\programs\python\python36\lib\site-packages\ipykernel-4.8.2-py3.6.egg\ipykernel_launcher.py:9: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  if __name__ == '__main__':
